### MODELOS SEÇÃO 6
Determinância e Política Monetária no Brasil: Uma Avaliação Empírica do Princípio de Taylor (1999–2024)

#### 0. SETUP

#### 0.1 Importar bibliotecas ####

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


#### 0.2 Importar dfs ####

##### ecobr #####

In [2]:
## df economia br 1999-2024
ecobr = pd.read_csv('../dados/fernando/economia_br9924.csv')
ecobr = ecobr.rename(columns={'Unnamed: 0': 'data'})
ecobr['data'] = pd.to_datetime(ecobr['data'], format='%Y-%m-%d')

In [3]:
ecobr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9437 entries, 0 to 9436
Data columns (total 6 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   data                          9437 non-null   datetime64[ns]
 1   IPCA_Mensal                   9437 non-null   float64       
 2   IPCA_12m                      9437 non-null   float64       
 3   Selic_Meta                    9434 non-null   float64       
 4   Expectativa_IPCA_AnoCorrente  9132 non-null   float64       
 5   PIB_Indice                    9437 non-null   float64       
dtypes: datetime64[ns](1), float64(5)
memory usage: 442.5 KB


In [4]:
# Renomear colunas
ecobr = ecobr.rename(columns={
    'data': 'ano_mes_dia',
    'ipca_mensal': 'ipca_mensal',
    'IPCA_12m': 'ipca12',
    'Selic_Meta': 'selic',
    'Expectativa_IPCA_AnoCorrente': 'ipca_expectativa',
    'PIB_Indice': 'pib_indice'
})

In [10]:
ecobr

,ano_mes_dia,IPCA_Mensal,ipca12,selic,ipca_expectativa,pib_indice
0,1999-01-01,0.70,1.65,NaN,NaN,99.26343
1,1999-02-01,1.05,2.24,NaN,NaN,99.26343
2,1999-03-01,1.10,3.02,NaN,NaN,99.26343
3,1999-03-05,1.10,3.02,45.00,NaN,99.26343
4,1999-03-06,1.10,3.02,45.00,NaN,99.26343
...,...,...,...,...,...,...
9432,2024-12-27,0.52,4.83,12.25,4.867495,179.26343
9433,2024-12-28,0.52,4.83,12.25,4.867495,179.26343
9434,2024-12-29,0.52,4.83,12.25,4.867495,179.26343
9435,2024-12-30,0.52,4.83,12.25,4.867495,179.26343


In [5]:
ecobr.describe()

,ano_mes_dia,IPCA_Mensal,ipca12,selic,ipca_expectativa,pib_indice
count,9437,9437.000000,9437.000000,9434.000000,9132.000000,9437.000000
mean,2012-01-30 23:45:48.542969088,0.506562,6.254480,12.687063,5.625636,143.189876
min,1999-01-01 00:00:00,-0.680000,1.650000,2.000000,1.610952,99.263430
25%,2005-08-16 00:00:00,0.260000,4.480000,9.000000,4.242536,120.844476
50%,2012-01-31 00:00:00,0.460000,5.910000,12.250000,5.501495,153.657049
75%,2018-07-17 00:00:00,0.690000,7.350000,16.000000,6.424783,161.341778
max,2024-12-31 00:00:00,3.020000,17.240000,45.000000,12.406316,179.526155
std,NaN,0.394423,2.721792,5.548323,1.925875,23.252670


##### hiato #####

In [6]:
## df economia br 1999-2024
hiato = pd.read_csv('../dados/fernando/hiato_br9924.csv')
hiato = hiato.drop(columns=['Unnamed: 0'])
hiato = hiato.rename(columns={'data': 'ano_trim'})

# Função para converter ano_trim para data no formato desejado
def ano_trim_to_date(ano_trim):
    ano = int(ano_trim[:4])
    trimestre = int(ano_trim[-1])
    if trimestre == 1:
        return f"{ano}-03-31"
    elif trimestre == 2:
        return f"{ano}-06-30"
    elif trimestre == 3:
        return f"{ano}-09-30"
    elif trimestre == 4:
        return f"{ano}-12-31"

hiato['data'] = hiato['ano_trim'].apply(ano_trim_to_date)
hiato['data'] = pd.to_datetime(hiato['data'], format='%Y-%m-%d')

# Rearranje a coluna 'data' para ficar ao lado direito da coluna 'ano_trim'
cols = hiato.columns.tolist()
cols.insert(1, cols.pop(cols.index('data')))
hiato = hiato[cols]


In [7]:
hiato.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104 entries, 0 to 103
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   ano_trim              104 non-null    object        
 1   data                  104 non-null    datetime64[ns]
 2   Tendência Linear      104 non-null    float64       
 3   Tendência Quadrática  104 non-null    float64       
 4   Filtro HP             104 non-null    float64       
 5   Filtro de Hamilton    93 non-null     float64       
 6   IFI                   104 non-null    float64       
 7   BCB                   85 non-null     float64       
dtypes: datetime64[ns](1), float64(6), object(1)
memory usage: 6.6+ KB


In [8]:
# Renomear colunas
hiato = hiato.rename(columns={
    'data': 'ano_mes_dia',
    'Tendência Linear': 'tendencia_linear',
    'Tendência Quadrática': 'tendencia_quadratica',
    'Filtro HP': 'filtro_hp',
    'Filtro de Hamilton': 'filtro_hamilton',
    'IFI': 'ifi',
    'BCB': 'bcb'
})

In [9]:
hiato

,ano_trim,ano_mes_dia,tendencia_linear,tendencia_quadratica,filtro_hp,filtro_hamilton,ifi,bcb
0,1999Q1,1999-03-31,-7.033520,3.771221,0.003118,NaN,-1.51,NaN
1,1999Q2,1999-06-30,-7.051331,3.088954,-0.154229,NaN,-1.81,NaN
2,1999Q3,1999-09-30,-7.558478,1.884706,-0.836256,NaN,-1.95,NaN
3,1999Q4,1999-12-31,-6.748484,2.146928,-0.105525,NaN,-1.80,NaN
4,2000Q1,2000-03-31,-6.259466,2.065477,0.280346,NaN,-1.29,NaN
...,...,...,...,...,...,...,...,...
99,2023Q4,2023-12-31,-4.551327,3.925313,0.069892,3.798547,-0.13,0.47
100,2024Q1,2024-03-31,-4.298396,4.830733,0.132524,3.952841,0.10,0.71
101,2024Q2,2024-06-30,-3.188127,6.701502,1.086839,4.496560,0.47,0.95
102,2024Q3,2024-09-30,-2.844051,7.755229,1.237231,4.589841,0.91,1.14


In [11]:
hiato.describe()

,ano_mes_dia,tendencia_linear,tendencia_quadratica,filtro_hp,filtro_hamilton,ifi,bcb
count,104,104.000000,104.000000,104.000000,93.000000,104.000000,85.000000
mean,2012-02-14 06:00:00,0.214262,0.083480,0.016705,0.090397,-0.819423,-0.264706
min,1999-03-31 00:00:00,-14.782081,-13.591256,-9.789709,-11.282942,-9.570000,-6.740000
25%,2005-09-07 00:00:00,-4.953473,-3.266832,-0.738433,-1.967810,-1.700000,-1.520000
50%,2012-02-14 12:00:00,-2.016555,-0.827425,0.339929,0.290108,-0.700000,0.160000
75%,2018-07-23 00:00:00,5.143897,3.660817,1.077431,2.432698,0.460000,0.960000
max,2024-12-31 00:00:00,12.841388,7.755229,3.548670,10.856603,2.150000,2.400000
std,NaN,6.663592,4.102126,1.816306,4.252716,1.728831,1.697518


#### 0.3 Construir df ####

##### Equalização períodos (tudo trimestral) #####

 ##### Juntar dfs (ecobr e hiato) #####